# DI 501 - Introduction to Data Informatics 
### Intro to data exploration (pandas, matplotlib, seaborn, scipy)

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
import statsmodels.api as sm 
from scipy.stats import spearmanr, pearsonr
from statsmodels.stats.multitest import multipletests
pd.set_option('display.max_columns', None)


## Loading data

To easily locate files you can put them in the same folder as the notebook file. Otherwise enter the path to the file.

In [ ]:
data = pd.read_csv('2024 QS World University Rankings 1.1 (For qs.com).csv')
data.head()

Sometimes default parameters in functions don't work for us. Here, we need to change the seperator using the `sep` argument.

In [ ]:
data = pd.read_csv('2024 QS World University Rankings 1.1 (For qs.com).csv', sep=';')
data.head()

Here we see some columns at the top are not needed. To look at all the arguments a function has, and get more info about it we can append or prepend `?` to the function call, or visit the defined [page](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) in the package webpage/repository.

In [ ]:
pd.read_csv?

We can see we have an argument named `skiprows`. Lets change it to skip the first 3 rows.

In [ ]:
data = pd.read_csv('2024 QS World University Rankings 1.1 (For qs.com).csv', sep=';', skiprows=3)
data.head(3)

All looks good. We have columns with names, acronyms, and numbers. We can see all column names by accessing the `columns` attribute of the pandas dataframe.

### Unifying column names

In [ ]:
data.columns

Notice how some columns are all uppercase, and others have each word capitalized. For further analysis it is beneficial to convert all column names to lowercase strings.

They also have spaces which will make referring to them cumbersome, so we will replace them with an underscore as well.

In [ ]:
data.columns = [x.lower().replace(' ', '_') for x in data.columns]

data.columns

But these acronyms don't mean much. Below is a list of explanations for most of the rows compiled from various pages from QS webpages.

## QS World University Rankings Explained

* **Academic reputation** – Accounting for 30 per cent of the overall score, academic reputation looks at the teaching and research quality at the world’s universities. We collate over 130,000 expert opinions from the higher education space, creating the largest survey of academic opinion in the world.  

* **Employer reputation** – We know that students want to graduate with the skills and knowledge required for the employment market. We assess how institutions prepare students for successful careers, and which institutions provide the most competent, innovative, and effective graduates.  

* **Faculty/student ratio** – This indicator recognises that a high number of academics per student reduces the teaching burden and creates a more supportive student experience. We assess how institutions provide students with meaningful access to lecturers and tutors.  

* **Citations per faculty** – We measure university research quality with a citation per faculty metric, taking the total number of academic citations in papers produced by a university in a five-year period. 

* **International student ratio & International faculty ratio** – A highly international university creates a number of benefits. It demonstrates the ability to attract quality students and staff from across the world, and it implies a highly global outlook. Strong international institutions provide a multinational environment, building international sympathies and global awareness.

* **International research network** - IRN Index reflects the ability of institutions to diversify the geography of their international research network by establishing sustainable research partnerships with other higher education institutions.

* **Graduate employment rate** - Defined as the percentage of graduates who go on to paid (non-voluntary) work within 15 months of finishing their degree. We consider any mode of employment (full-time or part-time), even if unknown.

* **Sustainability** - Measures which institutions are demonstrating a commitment to a more sustainable existence. More than just the commitment, it looks for outwards evidence of this - from the impact that alumni are making in science and technology to solve climate issues, to the impact of research being done across the UN's 17 sustainable development goals.  It evaluates the social and environmental impact of universities as a center's of education and research, as well as a major employers with the operational sustainability challenges of any large and complex organization.


### Weights for overall rank

| Performances Lenses            | 2024 Edition Weights  | Change from previous editions |
|--------------------------------|-----------------------|-------------------------------|
| Academic Reputation            | 30%                   | 10% deducted                  |
| Employer Reputation            | 15%                   | 5% added                      |
| Faculty Student Ratio          | 10%                   | 10% deducted                  |
| Citations per Faculty          | 20%                   | No change                     |
| International Faculty Ratio    | 5%                    | No change                     |
| International Student Ratio    | 5%                    | No change                     |
| International Research Network | 5%                    | New                           |
| Employment Outcomes            | 5%                    | New                           |
| Sustainability                 | 5%                    | New                           |

### Classification columns

<table>

<td>

|    | Size        | Students         |
|----|-------------|------------------|
| XL | Extra Large | >=30,000 |
| L  | Large       | >=12,000         |
| M  | Medium      | >=5,000          |
| S  | Small       | <5,000 |

</td>
<td>

|    | Focus              | Faculty Area                         |
|----|--------------------|--------------------------------------|
| FC | Full comprehensive | All 5 faculty areas + medical school |
| CO | Comprehensive      | All 5 faculty areas                  |
| FO | Focused            | 3 or 4 faculty areas                 |
| SP | Specialist         | 2 or fewer faculty areas             |

</td>
<td>

|   | Classification | Age                    |
|---|----------------|------------------------|
| 5 | Historic       | >=100 years old        |
| 4 | Mature         | 50-99 years old        |
| 3 | Established    | 25-49 years old        |
| 2 | Young          | 10-24 years old        |
| 1 | New            | < 10 years old |

</td>
<td>

|    | Research Intensity |
|----|--------------------|
| VH | Very High          |
| HI | High               |
| MD | Medium             |
| LO | Low                |

</td>
</table>


Sources: <br>
https://www.topuniversities.com/qs-world-university-rankings/methodology <br>
https://support.qs.com/hc/en-gb/articles/4405955370898-QS-World-University-Rankings <br>
https://support.qs.com/hc/en-gb/articles/6107352412828-QS-World-University-Rankings-Sustainability- <br>
https://support.qs.com/hc/en-gb/articles/6478203732380-2024-Rankings-Cycle <br>
https://support.qs.com/hc/en-gb/articles/360021876820-QS-Institution-Classifications

## Initial exploration

Before delving into details, we can quickly have an idea about our dataset with these two methods:

* `.info()` method shows information like how many rows and columns there are, how many non NaN values exist, which object types python chose for each column etc. 

In [ ]:
data.info(verbose=True)

* `.describe()` method meanwhile calculates descriptive statistics for numeric columns.

In [ ]:
data.describe()

Notice how only numeric attributes were considered for this function. Although for columns like 'age band' these metrics don't make sense as it is an **ordinal** attribute.

Let's focus on turkish universities for a bit. We can subset the dataset like so:

In [ ]:
turkey = data[data['location_code'] == 'TR']
turkey.head()

Another way to do the same thing is to use the query method. Note: we would've needed need backticks for variable names with spaces, but we avoided that with the underscores.

In [ ]:
data.query('location_code == "TR"').head()

In [ ]:
plt.hist(turkey['ger_score'])

We can see the histogram function returns two arrays and a plot. One array is the bar heights and the other is the bin end point. 

But of course we can use different arguments to change how many bins there are, or how much the binwidth should be.

In [ ]:
data.hist(figsize=(10,10), bins=30);
plt.suptitle('Histograms of all the numeric variables', y=0.95, fontsize=20, fontweight='semibold');

Returning to the distribution of graduate employment rate for Turkey we can make custom graphs using matplotlib and an extension to it, **seaborn**. 

Notice how the `kde` parameter added a kernel estimated density function on top.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,5))
fig.suptitle('Same plot from different libraries', size=15)

axes[0].set_xlabel('ger_score')
axes[0].set_ylabel('Count')
axes[0].set_title('matplotlib histogram')
axes[0].hist(x=turkey['ger_score'], bins=30);

axes[1].set_title('Seaborn histplot')
sns.histplot(turkey['ger_score'], kde=True, bins=30, ax=axes[1]);

As you may recall we had rank columns identified as an "object" and the scores as "float64". 

In [ ]:
data.columns

In [ ]:
import re
rank_cols = {x for x in data.columns if x.endswith('rank')}
rank_cols2 = {x for x in data.columns if re.findall('rank', x)}

# assert rank_cols == rank_cols2



In [ ]:
rank_cols2-rank_cols

While this was a good exercise we actually want to keep the rank display columns as they give information on how universities changed rank from 2023 to 2024.

### Fixing rank columns

As you may recall we had rank columns identified as an "object" and the scores as "float64". We can easily recover rank from the score by sorting, so we will remove them.

In [ ]:
simplified = data.iloc[:,~data.columns.isin(rank_cols)]

simplified.info()

### **Question 1)**

Using pandas series method [`str.extract`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html) and an appropriate [regular expression](https://images.datacamp.com/image/upload/v1665049611/Marketing/Blog/Regular_Expressions_Cheat_Sheet.pdf) pattern convert the `rank_display` column to pure integers by extracting the digits.

In [ ]:
# fill here #

In [ ]:
simplified.dtypes

In [ ]:
simplified.rank_display = simplified.rank_display.astype('int')

simplified.rank_display2 = simplified.rank_display2.astype('float')

### **Question 2)**

In the above code, try converting `rank_display2` to an integer. If you encounter an error, analyze why `rank_display2` was cast as a float while rank_display was successfully converted to an integer.

In [ ]:
# fill here #

> Fill here by double clicking me!

### Fixing overall score column

In [ ]:
simplified['overall_score'].unique()

In [ ]:
simplified.overall_score = simplified['overall_score'].replace('-', np.NaN).astype(float)

### Fixing age band column

In [ ]:
simplified['age_band'].unique()

In [ ]:
simplified.loc[:,'age_band'] = simplified['age_band'].astype('object').values

## Location Based Inference

We might get tempted to compare global trends using countries, but we have more than 100 unique countries listed here.

In [ ]:
len(simplified.location.unique())

Most of our instances come from the US, UK and China.

In [ ]:
simplified.location.value_counts()

We can access the raw numbers inside a pandas series (shown above) by accessing their values. 

This highlights the fact that pandas dataframes are wrapped around numpy arrays.

In [ ]:
simplified.location.value_counts().values

Instead of country based aggregation, we can add a higher order category, like which continent that country belongs to, and then we can group the data based on continent and compare inter-continent countries, or we can compare continents themselves.

You'd need to uncomment the lines below and run them if you don't have the pycountry package in your environment.

In [ ]:
# !pip install pycountry
# !pip install pycountry_convert

In [ ]:
import pycountry
from pycountry_convert import country_alpha2_to_continent_code

In [ ]:
country_alpha2_to_continent_code('TR')

This function is basically a dictionary where each country's abbreviation is mapped to a continent.

The library is more extensive than this though, as it covers countries official names, and even flags with emojis.

In [ ]:
pycountry.countries.get(name='United Kingdom')

If we were to apply this function to our location code column we would get an error because the United Kingdom is coded as UK in this dataset.

In [ ]:
simplified.iloc[1,:5]

We can replace instances of UK with GB using the replace method of pandas dataframes.

In [ ]:
simplified.loc[:, 'location_code'] = simplified['location_code'].replace('UK', 'GB')

Now we can apply our function to our location code column to get continents.

In [ ]:
simplified.loc[:,'continent'] = simplified['location_code'].apply(country_alpha2_to_continent_code)

In [ ]:
simplified[['continent', 'location_code']].head()

Small note about violin plots: the kernel bandwidth parameter `bw` determines how smooth or rugged the graph is.

In [ ]:
bw = np.linspace(0.05,0.5,6)

fig, axes = plt.subplots(2,3,figsize=(9,6))
fig.suptitle('Effect of changing bandwith on a violin plot')

for i,j in zip(axes.flatten(),bw):
    sns.violinplot(x=simplified['ar_score'], ax=i, bw_method=j)
    i.set_xlabel('')
    i.set_title(f'bw = {np.round(j,2)}')

For large datasets (thousands of observations): Use a small bw (e.g., 0.05-0.2) to reveal finer details.<br>
For small datasets: Use a larger bw (e.g., 0.2-0.3) for a cleaner, more general shape.<br>
A good starting point: bw=0.2 or using "scott" for automatic selection.

### Inference from data

Now let's use our new continent variable to aggregate data and show various variables' spreads.

As a side note we can use different themes for our matplotlib plots. More can be found [here](https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html).

In [ ]:
plt.style.use('fivethirtyeight')


In [ ]:
sns.violinplot(data=simplified, x='ar_score', y='continent', 
               hue='continent', bw_method=0.05)

There's a trend of high academic reputation among north american and oceanean universities, with long extending tails going up a bit towards the 100 score mark.

In [ ]:
plt.style.use('default')

In [ ]:
g = sns.FacetGrid(simplified, col="continent",
                  hue="continent", col_wrap=3)

g.map_dataframe(sns.histplot, x="ger_score");

On the other hand there were quite a bit of universities with high graduate employment rate in Africa and Oceania that isn't as prevalent in other continents.

With this we can also subset our data such that only countries from a certain continent are present. Then we can look at individual countries within a continent and have a concise grahp as opposed to all 100 countries being plotted at once.

In [ ]:
fig = plt.figure(figsize=(15,9))

sns.violinplot(data=simplified.query('continent == "EU"'), 
               x='isr_score', y='location', bw_method=0.05);

We can see that countries like Croatia, Romania, and Serbia generally don't have many international students, whereas countries like the UK, Switzerland and Austria generally have a lot of international students. This can suggest people who pick these high ranked universities have preferences for some countries over others.

Since the plot is so tightly packed, only the boxplot in the middle of the violins show up, but it is a neat enough graphic to include.

We can also subset our data such that only universities from a few countries of our choosing exist within it. Here the tuples of binary arrays that locate where TR and PL universities are combined with the `OR` function such that if either of the rows have these two countries it will get picked up by our final outer subset.

In [ ]:
# trpl = simplified[(simplified['location_code'] == 'TR') | (simplified['location_code'] == 'PL') ]

trpl = simplified.query('location_code == "TR" or location_code == "PL"')

In [ ]:
sns.violinplot(data=trpl, x='isr_score', y='location_code', bw_method=0.2)

Looks like there is an outlier (or two) for Turkey in international student ratio. Let's identify which universities these are.

### **Question 3)** 
How can we go about finding outliers in this subset of the data? Experiment with at least two different methods to detect outliers, compare their effectiveness, and justify your findings.  You can use [loc[]](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html) or [query()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html) to filter and display the dataset.


In [ ]:
#fill here#

### **Question 4)**

Out of all historic universities with more than 30k students but are not "fully comprehensive" or "comprehensive" in their subject coverage, which ones have the best sustainability score according to the QS? Use the [query()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html) method to filter universities.

In [ ]:
# fill here #

We can then start looking for any columns with a symmetric distribution. Since most of the statistical analyses we will cover rely on the assumption of normality, this is an important step, although normality should be checked with statistical tests like shapiro-wilk and similar tests.

Here Quantiles-Quantiles plots of 4 variables chosen is drawn. The `line` parameter is set to "s", which tries to find a best fit line, and if none is found, it will not draw a line. Most variables chosen have a very long tail on the right, so it makes sense that many of the plots reflect this. irn score looks a bit different because it's distribution closeley resembles the uniform distribution.

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(10,10))

vars = ['ar_score', 'ger_score', 'irn_score', 'isr_score']

for i,j in zip(axes.flatten(), vars):
  sm.qqplot(simplified[j], line="q", ax=i);
  i.set_title(f'{j}')
  

Just to see how it is done, let's check the normality of ar score using the shapiro wilk test.

In [ ]:
stat, p = stats.shapiro(simplified['ar_score'].dropna())

print(f'Shapiro wilk statistic: {stat}\np value: {p}')

We can easily reject the hypothesis that the values were picked from a normal distribution.

### Column to column relationships

Now that we can have a deep understanding of each column, let's see if combinations of any two column contains further information. <br>

`sns.pairplot` function does this well. <br>

Figsave has a parameter called `dpi` that changes the resolution of the output graphic. Higher dpi like 300 here implies large resolution and large file sizes. 

***Make sure your graphics have appropriate colors for visibility, no overlaps, and a big figure size so details can be made out.***

In [ ]:
sns.pairplot(simplified.sample(frac=0.3),
            corner = True, height=1.3, aspect=1.3)
# plt.savefig('plot.png', dpi=300)

### **Question 5)** 
What patterns do you observe in the scatter plots? Do any variables show a strong positive or negative correlation?

> Fill here by double clicking me!

### Correlations of scores with overall score

For this final part lets look at a chunk of these correlations against score only. We can compare these numbers by the percentages given in the beginning of this notebook.

We can also compute correlations with the corr method, and the default is pearson's r, however, we will use Spearman correlation instead of Pearson because the variables do not follow a normal distribution.
Correlations can only be computed for numeric attributes, so we need to subset the dataset with those columns.

In [ ]:
num_cols = [col for col,dtype in zip(simplified.columns, simplified.dtypes) if dtype!='object']
num_cols

We should remove rows with NaN values in relevant columns.

In [ ]:
simplified = simplified.dropna(subset=['overall_score'] + num_cols[2:-1])

In [ ]:
corrs = []
p_values = []
for col in num_cols[2:-1]:
    corr, p_val = spearmanr(simplified[col], simplified['overall_score'])
    corrs.append(corr)
    p_values.append(p_val)

corr_df = pd.DataFrame({'Variable': num_cols[2:-1], 'Spearman Correlation': corrs, 'p-value': p_values})
corr_df = corr_df.sort_values('Spearman Correlation', ascending=False).reset_index(drop=True)

corr_df



## P-Value Issue and Multiple Testing Correction
### **Question 6)** 
If we conduct multiple statistical tests on different variables, how does this affect the reliability of our p-values?



> Fill here by double clicking me!

 First, we apply the Bonferroni correction.

In [ ]:
corrected_pvals = multipletests(corr_df['p-value'], method='bonferroni')[1]
corr_df['Bonferroni Corrected p-value'] = corrected_pvals


However, Bonferroni correction is very strict, increasing the risk of Type II errors. As an alternative, we also apply the Benjamini-Hochberg (FDR) and Holm corrections.

In [ ]:
bh_corrected_pvals = multipletests(corr_df['p-value'], method='fdr_bh')[1]
holm_corrected_pvals = multipletests(corr_df['p-value'], method='holm')[1]

corr_df['Benjamini-Hochberg Corrected p-value'] = bh_corrected_pvals
corr_df['Holm Corrected p-value'] = holm_corrected_pvals
corr_df




1. Strong correlation (Absolute Spearman Correlation	 > 0.6)
   - `ar_score` (0.85), `er_score` (0.67), `sus_score` (0.65), `ger_score` (0.65)  
   - These variables have a strong positive monotonic relationship with `overall_score`.  
   - Higher values in these variables strongly correspond to higher `overall_score`. <br> <br>
2. Weak correlation (Absolute Spearman Correlation	 < 0.3)  
   - `fsr_score` (0.27)  
   - This variable has the weakest correlation with `overall_score`.  
   - Changes in `fsr_score` have little impact on the overall score.<br><br>

3. Statistical significance (p-value)
   - All p-values are extremely low (< 10⁻¹⁰), meaning all correlations are highly statistically significant.  
   - This confirms that these correlations are not due to random chance.  


# Comparison with Assigned Percentages
Predefined percentage values are used to compare with observed correlations. One thing to realize here is we are using the reset_index method to save the index as another column. This is then used to color each bar in the barplot below.

In [ ]:

perc = pd.Series([0.3, 0.15, 0.1, 0.2, 0.05, 0.05, 0.05, 0.05, 0.05], index=num_cols[2:-1])
perc_df = perc.sort_index().reset_index()
perc_df.columns = ['Variable', 'Percentage']


In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle('Spearman Correlation with Overall Score and Assigned Percentages', size=15, y=1)

ordered_variables = perc_df['Variable'].tolist()
corr_df = corr_df.set_index('Variable').loc[ordered_variables].reset_index()

sns.barplot(y='Variable', x='Spearman Correlation', orient='h', data=corr_df, ax=axes[0], hue='Variable', palette="viridis", legend=False)
axes[0].set_xlabel('Spearman Correlation')
axes[0].set_ylabel('')
axes[0].set_title('Correlation with Overall Score')


sns.barplot(y='Variable', x='Percentage', orient='h', data=perc_df, ax=axes[1], hue='Variable', palette="viridis", legend=False)
axes[1].set_xlabel('Assigned Percentage')
axes[1].set_ylabel('')
axes[1].set_title('Assigned Percentages')

plt.tight_layout()
plt.show()


Despite the claim of the authors, we find some columns like sus score, ger score and irn score to highly correlate with overall score. This doesn't automatically mean there is an additive effect though. For that we'd need to create a regression model and check how much each attribute contributes to the variation of the score.